In [214]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
import time
import json
import pandas as pd
    
driver = webdriver.Chrome()
driver.get("https://apprendre.tv5monde.com/fr/tcf/simulation-du-tcf#tcf_header")

wait = WebDriverWait(driver, 10)
iframe_element = driver.find_element(By.XPATH, '//iframe[@name="framecontent"]')
iframe_src = iframe_element.get_attribute("src")

#extracting audio_urls
script_element = driver.find_element(By.XPATH,'//script[@type="application/json"]')

json_data = script_element.get_attribute('innerHTML')
data = json.loads(json_data)
mp3=data['audiofieldjplayer']['audiofield']['files']
number_url=[]
question_url=[]
second_question=[]
for index in range(len(mp3)):
    if index%2==0:
        number_url.append(mp3[index]['file'])
    
    else:
        question_url.append(mp3[index]['file'])
        
for i in range(50):
    number_url.append('None')
    question_url.append('None')
id_question=[]
question_type=[]
url_image=[]
all2=[]
main_question=[]
second_question=[]
    
for i in range(80):
    response = requests.get(iframe_src)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    

    #question ID 
    index = iframe_src.find("qid")
    id_question.append(iframe_src[index+4:])
    


    #question type
    qt = soup.find('div', {'class': 'tcf-skill'})
    question_type.append(qt.text.strip())

    #URL image
    image_element=soup.find('img',{'class':'img-responsive'})
    if (image_element is not None):
        src = image_element.get('src')
        image_url='https://apprendre.tv5monde.com/'+src
        url_image.append(image_url)
        
    else:
        url_image.append('None')
        

    #all available answers
    av_answers = soup.find_all('span', {'class': 'tcf-response-text'})
    all_answer=[]
    for answer in av_answers:
        all_answer.append(answer.text.strip())
    all2.append(all_answer)

    
    
    #main question
    mq= soup.find('div', {'class': 'tcf-consigne'})
    main_question.append(mq.text.strip())
    
    
    #second question
    sq= soup.find('div', {'class': 'tcf-question-wrapper'})
    second_question.append(sq.text.strip())
    
    
    
    
    
    #moving to the next page
    driver.get(iframe_src)
    time.sleep(0.5)
    href = driver.find_element(By.XPATH, '//*[@id="tcf_new_test"]/div[5]/div[1]/a')
    href_value = href.get_attribute('href')
    iframe_src = href_value

#extracting the correct answer

driver.get(iframe_src)
time.sleep(1)

div_element = driver.find_element(By.XPATH, 'html/body/div[2]/div/div/div[2]/div')
a_element = div_element.find_elements(By.TAG_NAME,'a')
href_correct_answers=[]
for element in a_element:
    href_correct_answers.append(element.get_attribute('href'))

correct_answers=[]

answer_choices=["choixA", "choixB", "choixC", "choixD"]
for element in href_correct_answers:
    driver.get(element)
    for choice in answer_choices:
            xpatht = f'//a[@class="tcf-choix-item {choice} tcf-choix-resultat tcf-choix-bonnereponse"]/span[2]'  
            try:
                correct= driver.find_element(By.XPATH,xpatht)
            except NoSuchElementException:
                pass
    correct_answers.append(correct.text) 




#importing data in a csv file
data = {
    'question_id': id_question,
    'question_type': question_type,
    'URL_image': url_image,
    'number_question_audio': number_url,
    'question_audio': question_url,
    'second_question':second_question,
    'main_question': main_question,
    'available_answers': all2,
    'correct_answers': correct_answers
}

df = pd.DataFrame(data)
excel_file = 'data_finaltcf3.xlsx'
sheet_name = 'tcf'
df.to_excel(excel_file, sheet_name=sheet_name, index=False)
driver.quit()